<a href="https://colab.research.google.com/github/Harooniqbal4879/AgenticAI/blob/main/Capstone2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
# 1. Installthe required
!pip install openai serpapi requests python-dotenv
!pip install pyngrok
!pip install openai serpapi requests python-dotenv pyngrok

import os
import requests
from openai import OpenAI
from google.colab import userdata # Import userdata
from pyngrok import ngrok # from google.colab import userdata # Keep this line commented out if using direct token
import uuid # For generating state
import http.server
import socketserver
import threading


In [26]:
# --- Step 1.5: Start a simple HTTP server for ngrok to connect to ---
# This server will just acknowledge requests, as the main logic is in the Colab notebook

PORT = 8000

class Handler(http.server.SimpleHTTPRequestHandler):
    def do_GET(self):
        # This is where LinkedIn will redirect with the authorization code
        # In a real application, you would parse the 'code' and 'state' from self.path
        # For this notebook, we just need the server to run so ngrok can connect
        print(f"Received request on {self.path}")
        self.send_response(200)
        self.send_header("Content-type", "text/html")
        self.end_headers()
        self.wfile.write(b"<html><body><h1>Received request. Check Colab output for details.</h1></body></html>")

# Use threading to run the server in the background
def run_server():
    with socketserver.TCPServer(("", PORT), Handler) as httpd:
        print(f"Serving HTTP on port {PORT}")
        httpd.serve_forever()

# Run the server in a separate thread so the Colab notebook can continue
server_thread = threading.Thread(target=run_server)
server_thread.daemon = True  # Allow the main program to exit even if the thread is running
server_thread.start()

print("Simple HTTP server started in a background thread on port 8000.")
print("You can now run the ngrok cell.")

Simple HTTP server started in a background thread on port 8000.
You can now run the ngrok cell.


Exception in thread 

In [27]:
# from google.colab import userdata # Keep this line commented out if using direct token

# Get the ngrok authtoken (using the provided token directly for now)
# NGROK_AUTH_TOKEN = userdata.get("NGROK_AUTH_TOKEN") # Comment out if using direct token
NGROK_AUTH_TOKEN = "329d7PaznWat7gpdKBNidRNI3u1_4prNmGXkz1KothxLMXDMX" # Use the provided token

# Authenticate ngrok
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Expose port 8000
public_url = ngrok.connect(8000)
print("Public URL:", public_url)

Thread-7 (run_server):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipython-input-842775339.py", line 19, in run_server
  File "/usr/lib/python3.12/socketserver.py", line 457, in __init__
    self.server_bind()
  File "/usr/lib/python3.12/socketserver.py", line 478, in server_bind
    self.socket.bind(self.server_address)
OSError: [Errno 98] Address already in use


Public URL: NgrokTunnel: "https://63a132bfaf33.ngrok-free.app" -> "http://localhost:8000"


In [28]:
# Load API keys from Colab secrets
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
NGROK_AUTH_TOKEN = userdata.get("NGROK_AUTH_TOKEN")

# Initialize OpenAI Client
client = OpenAI(api_key=OPENAI_API_KEY)

# Set ngrok authtoken (using provided token directly for demonstration)
# For security, prefer storing in Colab secrets and using userdata.get()
ngrok.set_auth_token("329d7PaznWat7gpdKBNidRNI3u1_4prNmGXkz1KothxLMXDMX")

# Expose port 8000 using ngrok to get a public redirect URL
# You can change the port if your local server runs on a different one
try:
    public_url = ngrok.connect(8000)
    print("ngrok Public URL:", public_url)
    # Use this ngrok URL as your REDIRECT_URI in your LinkedIn Developer App settings
    # Convert PyngrokTunnel object to string and append /callback
    # REDIRECT_URI = str(public_url) + "/callback"
    REDIRECT_URI = public_url.public_url + "/callback"
    # Ensure the redirect URI in your LinkedIn app exactly matches this URL
    print(f"Use this as your Redirect URL in LinkedIn app settings: {REDIRECT_URI}")

except Exception as e:
    print(f"Error starting ngrok: {e}")
    print("Please ensure you have added your NGROK_AUTH_TOKEN to Colab secrets.")
    # Fallback placeholder - make sure this matches your LinkedIn app if ngrok fails
    REDIRECT_URI = "YOUR_NGROK_OR_OTHER_REDIRECT_URL"
    print(f"Using fallback REDIRECT_URI: {REDIRECT_URI}")


# Your LinkedIn Application Credentials
CLIENT_ID = "868ac47cwrbkde"
CLIENT_SECRET = "WPL_AP1.Jo7dDtzbbx5Cum1r.OIzirg=="

# The scope for posting content (w_member_social is required for sharing)
SCOPE = "w_member_social" # Use w_member_social for posting

# The state parameter is used to maintain state between the request and callback
STATE = str(uuid.uuid4())

# Generate the authorization URL
auth_url = f"https://www.linkedin.com/oauth/v2/authorization?response_type=code&client_id={CLIENT_ID}&redirect_uri={REDIRECT_URI}&state={STATE}&scope={SCOPE}"

print("\n--- Step 2: Authorize Application ---")
print("Please open the following URL in your browser and authorize your LinkedIn application:")
print(auth_url)

# After authorizing, LinkedIn will redirect you to your REDIRECT_URI with a 'code' and 'state' query parameter.
# You will need to manually capture the 'code' from the redirect URL in the next step.

ngrok Public URL: NgrokTunnel: "https://d470ad0be463.ngrok-free.app" -> "http://localhost:8000"
Use this as your Redirect URL in LinkedIn app settings: https://d470ad0be463.ngrok-free.app/callback

--- Step 2: Authorize Application ---
Please open the following URL in your browser and authorize your LinkedIn application:
https://www.linkedin.com/oauth/v2/authorization?response_type=code&client_id=868ac47cwrbkde&redirect_uri=https://d470ad0be463.ngrok-free.app/callback&state=de086106-3ffa-4c2a-bf08-0d7f783b68e2&scope=w_member_social


In [29]:
import requests

# Your LinkedIn app credentials
CLIENT_ID = "868ac47cwrbkde"
CLIENT_SECRET = "WPL_AP1.Jo7dDtzbbx5Cum1r.OIzirg=="
REDIRECT_URI = "https://0864f959e430.ngrok-free.app/callback"  # must match LinkedIn app settings
AUTH_CODE = "AQSW7Lrjvz0XsMdRegFspzf0zskv2w4EYUWdFGlqCQLauD8AJcTW4NDOEM7LvDvJIKMmddK-ECrxAjyA--QjdQBvyACWH_fvQv372yWyiO97BgTZzPP6kfjxcxcgZfIi3H1AwOcsGwbmquAuEoM2suBhi4uEewjNC-93-7zyx5P8Tx-pUzK-U5sp1wLAe3YrBwUE7iPaNaSYwlxBHps&state=a69bd3df-a561-4915-bb92-8d3fd58fb1e5"

# Exchange code for access token
token_url = "https://www.linkedin.com/oauth/v2/accessToken"
data = {
    "grant_type": "authorization_code",
    "code": AUTH_CODE,
    "redirect_uri": REDIRECT_URI,
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET
}

response = requests.post(token_url, data=data, headers={"Content-Type": "application/x-www-form-urlencoded"})
print(response.json())


{'error': 'invalid_request', 'error_description': 'Unable to retrieve access token: authorization code not found'}


In [30]:
# Paste the authorization code you got from the redirect URL here
AUTHORIZATION_CODE = "AQQ2b2olVzjhLvL6eZAIcMfiGXNge2tokVvWNVa2N0bS7LTBXVjDPiKMbK4oIP3ty2PNgjbpbMRWT00kDUI8iPQN0nlW390uQTpoCUUUjSUt_jc2sLcxkgbjYYcDHkYr_aaSRIm0SeYX-VaQxC6c7XWuYpKKzuFjMMpYTCSSSK6km_yLSGzKymdUSBwr6Jx1jDDecvdm_plSd2u4SR8&state=a69bd3df-a561-4915-bb92-8d3fd58fb1e5"

# LinkedIn Token Endpoint
TOKEN_URL = "https://www.linkedin.com/oauth/v2/accessToken"

# Ensure REDIRECT_URI is defined (should be from the previous cell)
# If running this cell independently, define REDIRECT_URI here:
# REDIRECT_URI = "https://your-ngrok-url/callback" # Replace with your ngrok public URL + /callback

# Prepare the data for the token exchange request
token_data = {
    "grant_type": "authorization_code",
    "code": AUTHORIZATION_CODE,
    "redirect_uri": REDIRECT_URI,
    "client_id": CLIENT_ID, # Ensure CLIENT_ID is defined (should be from previous cell)
    "client_secret": CLIENT_SECRET # Ensure CLIENT_SECRET is defined (should be from previous cell)
}

print("\n--- Step 4: Exchange Authorization Code for Access Token ---")
# Make the POST request to exchange the authorization code for an access token
try:
    response = requests.post(TOKEN_URL, data=token_data)
    token_response_data = response.json()

    # Check for errors
    if "error" in token_response_data:
        print("Error exchanging authorization code for access token:")
        print(token_response_data)
        ACCESS_TOKEN = None
    else:
        ACCESS_TOKEN = token_response_data.get("access_token")
        expires_in = token_response_data.get("expires_in")
        print("Successfully obtained Access Token:")
        print(f"Access Token: {ACCESS_TOKEN[:10]}...") # Print truncated token for security
        print(f"Expires in: {expires_in} seconds")

except Exception as e:
    print(f"Error during token exchange: {e}")
    ACCESS_TOKEN = None


--- Step 4: Exchange Authorization Code for Access Token ---
Error exchanging authorization code for access token:
{'error': 'invalid_request', 'error_description': 'Unable to retrieve access token: authorization code not found'}


In [37]:
# Load env vars (you already set OPENAI_API_KEY, SERPAPI_API_KEY in Colab environment)
# client = OpenAI(api_key=os.getenv("OPENAI_API_KEY")) # Original line
client = OpenAI(api_key=userdata.get("OPENAI_API_KEY")) # Use userdata to get the key

# --- Step 2: Blog Generation Function ---
def generate_blog(content_idea: str):
    """Generates a LinkedIn blog post using OpenAI."""
    prompt = f"""
    You are an expert LinkedIn content writer.
    Write a professional, engaging LinkedIn blog post from the following idea:
    {content_idea}

    Make sure to:
    - Start with a hook
    - Use short paragraphs
    - Add bullet points where helpful
    - Include a clear call-to-action
    - Suggest relevant hashtags
    """

    response = client.chat.completions.create(
        model="gpt-4o",  # using GPT-4o
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7
    )

    return response.choices[0].message.content

# --- Step 3: Placeholder LinkedIn Posting Function ---
def post_to_linkedin(blog_text: str):
    """
    Placeholder for LinkedIn API Posting.
    You need:
    - LinkedIn Developer App
    - Client ID, Client Secret
    - Access Token with w_member_social scope
    """
    print("Simulating LinkedIn Post...")
    print("Post Content:")
    print(blog_text[:500], "...")  # preview first 500 chars
    print("\n[LinkedIn API credentials required to post automatically]")

# --- Step 4: Execution Example ---
print("\n--- Step 4: Generate and Simulate Posting Blog ---")
# Get content idea from user input
content_idea = input("Enter the content idea for the LinkedIn blog post: ")

blog_post = generate_blog(content_idea)

# Show blog
print("\n=== Generated LinkedIn Blog ===")
print(blog_post)

# Simulated LinkedIn posting
post_to_linkedin(blog_post)


--- Step 4: Generate and Simulate Posting Blog ---
Enter the content idea for the LinkedIn blog post: Is the AI future of business?

=== Generated LinkedIn Blog ===
**The AI Future of Business: Are We Ready?**

Imagine a world where your coffee maker knows your morning routine better than you do, where your work emails are drafted before you even think about them, and where every decision is backed by data-driven insights. This isn’t a scene from a sci-fi movie—it's the rapidly approaching future of business, powered by Artificial Intelligence (AI).

**Why AI Matters**

AI is no longer a futuristic concept; it’s a present reality that’s reshaping industries and redefining the way we work. From streamlining operations to enhancing customer experiences, the impact of AI is profound and pervasive.

- **Efficiency and Productivity**: AI automates repetitive tasks, freeing up valuable time for employees to focus on strategic initiatives.
- **Data-Driven Decisions**: With AI, businesses can

In [32]:
# --- Step 5: Blog Generation Function (from original code) ---
def generate_blog(content_idea: str):
    prompt = f"""
    You are an expert LinkedIn content writer.
    Write a professional, engaging LinkedIn blog post from the following idea:
    {content_idea}

    Make sure to:
    - Start with a hook
    - Use short paragraphs
    - Add bullet points where helpful
    - Include a clear call-to-action
    - Suggest relevant hashtags
    """

    response = client.chat.completions.create(
        model="gpt-4o",  # using GPT-4o
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7
    )

    return response.choices[0].message.content

# --- Step 6: Generate Blog Content ---
content_idea = "How Agentic AI systems are transforming business workflows"
blog_post = generate_blog(content_idea)

# Show blog
print("--- Generated LinkedIn Blog ---")
print(blog_post)

--- Generated LinkedIn Blog ---
🚀 **The Future is Here: How Agentic AI Systems Are Transforming Business Workflows**

Imagine a world where your business processes operate with the efficiency of a well-oiled machine, predicting challenges and seizing opportunities before they even arise. Welcome to the era of agentic AI systems—a revolutionary force that is reshaping how businesses function, making operations smarter, faster, and more efficient than ever before.

### The Power of Agentic AI

Agentic AI systems are not just tools; they are intelligent agents capable of independent action. They analyze data, learn patterns, and make decisions without constant human intervention. This capability is transforming business workflows across industries.

### Why Businesses Are Embracing Agentic AI

- **Enhanced Efficiency**: By automating routine tasks, agentic AI liberates human resources, allowing employees to focus on strategic initiatives.
  
- **Improved Decision-Making**: With real-time 

In [33]:
if ACCESS_TOKEN:
    # LinkedIn Sharing API Endpoint
    POST_URL = "https://api.linkedin.com/v2/ugcPosts"

    # Get the URN of the authenticated user
    profile_url = "https://api.linkedin.com/v2/me"
    headers = {
        "Authorization": f"Bearer {ACCESS_TOKEN}",
        "X-Restli-Protocol-Version": "2.0.0"
    }
    try:
        profile_response = requests.get(profile_url, headers=headers)
        profile_response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
        profile_data = profile_response.json()
        # The person URN is typically in the 'id' field for the '/v2/me' endpoint
        author_urn = f"urn:linkedin:person:{profile_data.get('id')}"
        print(f"Obtained Author URN: {author_urn}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching user profile: {e}")
        author_urn = None
        print("Could not retrieve author URN. Cannot post to LinkedIn.")


    if author_urn:
        # Prepare the post data
        post_data = {
            "author": author_urn,
            "lifecycleState": "PUBLISHED",
            "specificContent": {
                "com.linkedin.ugc.ShareContent": {
                    "shareCommentary": {
                        "text": blog_post # Use the blog_post generated earlier
                    },
                    "shareMediaCategory": "NONE" # Or "ARTICLE", "IMAGE", etc.
                }
            },
            "visibility": {
                "com.linkedin.ugc.MemberNetworkVisibility": "PUBLIC"
            }
        }

        # Make the POST request to publish the blog
        headers = {
            "Authorization": f"Bearer {ACCESS_TOKEN}",
            "Content-Type": "application/json",
            "X-Restli-Protocol-Version": "2.0.0"
        }
        print("\n--- Step 8: Publishing Blog Post ---")
        try:
            post_response = requests.post(POST_URL, json=post_data, headers=headers)
            post_response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)

            if post_response.status_code == 201:
                print("Blog post published successfully!")
                # The response contains the ID of the created post
                print("Post Response:", post_response.json())
            else:
                 # This part might be redundant due to raise_for_status, but kept for clarity
                print(f"Error publishing blog post. Status code: {post_response.status_code}")
                print("Response:", post_response.text)

        except requests.exceptions.RequestException as e:
            print(f"Error publishing blog post: {e}")


else:
    print("Access token not available. Cannot post to LinkedIn.")
    print("Please ensure you completed the authorization steps and obtained an access token.")

Access token not available. Cannot post to LinkedIn.
Please ensure you completed the authorization steps and obtained an access token.


In [34]:
# Query Handler Agent: Routes requests to appropriate specialized agents

def route_query(query: str, client):
    """
    Routes a user query to a hypothetical specialized agent based on its content.
    Uses an LLM to categorize the query.
    """
    prompt = f"""
    Analyze the following user query and determine the primary task or intent.
    Suggest which type of specialized agent or action would be best suited to handle it.
    Possible categories/agents include:
    - LinkedIn Posting (for requests related to creating or publishing LinkedIn content)
    - Data Analysis (for requests involving data manipulation, analysis, or visualization)
    - Code Generation (for requests asking for new code snippets)
    - Code Explanation (for requests asking to explain existing code)
    - API Interaction (for requests involving external APIs like LinkedIn, ngrok, etc.)
    - General Question (for general inquiries not fitting other categories)
    - Other (if none of the above fit well)

    Respond with ONLY the suggested category/agent name.

    Query: {query}
    """

    try:
        response = client.chat.completions.create(
            model="gpt-4o", # Using GPT-4o as it's capable of understanding intent
            messages=[{"role": "user", "content": prompt}],
            max_tokens=50, # Keep the response concise
            temperature=0.1 # Keep the response focused
        )
        suggested_agent = response.choices[0].message.content.strip()
        return suggested_agent

    except Exception as e:
        print(f"Error during query routing: {e}")
        return "Error Routing"

# Example Usage (assuming 'client' is initialized from a previous cell)
# if 'client' in globals():
#     user_query = "Generate a LinkedIn post about the benefits of cloud computing."
#     suggested_handler = route_query(user_query, client)
#     print(f"Query: '{user_query}'")
#     print(f"Suggested Handler: {suggested_handler}")

#     user_query_2 = "Explain the code in cell XYZ."
#     suggested_handler_2 = route_query(user_query_2, client)
#     print(f"Query: '{user_query_2}'")
#     print(f"Suggested Handler: {suggested_handler_2}")
# else:
#     print("OpenAI client not initialized. Cannot use query router.")

In [35]:
# Deep Research Agent: Conducts comprehensive web research and analysis

class DeepResearchAgent:
    """
    An agent designed to conduct comprehensive web research and analyze findings.
    """
    def __init__(self, client):
        self.client = client
        # Initialize any tools or resources needed for research here
        # e.g., search API client, scraping tools, analysis modules

    def conduct_research(self, query: str):
        """
        Conducts web research based on the query and analyzes the results.

        Args:
            query: The research query.

        Returns:
            A summary or structured analysis of the research findings.
        """
        print(f"Deep Research Agent: Conducting research for query: '{query}'")
        # Placeholder for web research logic
        # This would involve using search APIs (like SERP API),
        # potentially scraping, parsing information, and synthesizing it.
        # Example: Using a hypothetical search tool
        # search_results = self.search_tool.perform_search(query)
        # analysis = self.analyze_results(search_results)
        # return analysis

        return "Deep Research Agent: Research function placeholder."

    # def analyze_results(self, results):
    #     """
    #     Analyzes the raw research results.
    #     """
    #     # Placeholder for analysis logic
    #     pass

# Example Usage (assuming 'client' is initialized)
# if 'client' in globals():
#     research_agent = DeepResearchAgent(client)
#     research_findings = research_agent.conduct_research("latest trends in AI")
#     print(research_findings)

In [36]:
# SEO Blog Writer Agent: Creates search-optimized long-form content

class SEOBlogWriterAgent:
    """
    An agent designed to create search-optimized long-form blog content.
    """
    def __init__(self, client):
        self.client = client
        # Initialize any tools or resources needed for SEO analysis or content generation here
        # e.g., keyword research tools, SEO analysis modules

    def generate_seo_blog(self, topic: str, keywords: list = None):
        """
        Generates long-form blog content optimized for search engines.

        Args:
            topic: The main topic of the blog post.
            keywords: A list of keywords to incorporate for SEO.

        Returns:
            The generated SEO-optimized blog content.
        """
        print(f"SEO Blog Writer Agent: Generating blog post on '{topic}'...")
        if keywords:
            print(f"Optimizing for keywords: {', '.join(keywords)}")

        # Placeholder for advanced SEO content generation logic
        # This would involve:
        # 1. Using keywords to refine the prompt.
        # 2. Structuring the content with headings, subheadings.
        # 3. Incorporating keywords naturally throughout the text.
        # 4. Potentially analyzing generated text for SEO score.

        prompt = f"""
        Write a detailed, long-form blog post about "{topic}".
        """
        if keywords:
             prompt += f"\nEnsure the content is optimized for the following keywords: {', '.join(keywords)}."
             prompt += "\nStructure the blog with a clear introduction, body paragraphs, and a conclusion. Use headings and subheadings where appropriate."


        try:
            response = self.client.chat.completions.create(
                model="gpt-4o",  # Using GPT-4o for detailed content
                messages=[{"role": "user", "content": prompt}],
                temperature=0.7, # Allow for some creativity
                max_tokens=1500 # Allow for longer content
            )
            blog_content = response.choices[0].message.content.strip()
            return blog_content

        except Exception as e:
            print(f"Error during blog generation: {e}")
            return "SEO Blog Writer Agent: Blog generation failed."

# Example Usage (assuming 'client' is initialized)
# if 'client' in globals():
#     seo_writer = SEOBlogWriterAgent(client)
#     blog_post = seo_writer.generate_seo_blog(
#         topic="The Impact of AI on Marketing",
#         keywords=["AI in marketing", "marketing automation", "AI tools for marketing"]
#     )
#     print("--- Generated SEO Blog Post ---")
#     print(blog_post)

# Task
Set up a Streamlit chat interface to take user input.

## Install streamlit

### Subtask:
Add a code cell to install the Streamlit library.


**Reasoning**:
The subtask is to install the Streamlit library. This requires adding a new code cell with a pip install command.



In [38]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 67.6 MB/s eta 0:00:00


## Write streamlit app code

### Subtask:
Create a new code cell to write the Python code for the Streamlit application. This code will include setting up a basic chat interface to capture user input.


**Reasoning**:
The subtask requires creating a Streamlit application with a basic chat interface to capture user input. I need to write Python code in a new cell to achieve this, importing streamlit, setting a title, adding a text input and a button, and displaying the input when the button is clicked.



In [39]:
import streamlit as st

# Set the title of the application
st.title("Agentic AI Chat Interface")

# Create a text input field for user messages
user_input = st.text_input("Enter your message:")

# Add a button to send the message
send_button = st.button("Send")

# Check if the button is clicked and the input is not empty
if send_button and user_input:
    # Display the user's input
    st.info(f"You entered: {user_input}")


2025-09-02 23:13:35.863 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 23:13:36.015 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-09-02 23:13:36.016 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 23:13:36.017 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 23:13:36.019 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 23:13:36.021 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 23:13:36.023 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 23:13:36.024 Thread 'MainThread': mi

## Run streamlit app

### Subtask:
Add a code cell to run the Streamlit application. This will typically involve using a shell command like `streamlit run your_app.py`.


**Reasoning**:
Save the Streamlit app code to a file and then run it using the streamlit command.



In [40]:
# Save the Streamlit app code to a file named app.py
streamlit_code = """
import streamlit as st

# Set the title of the application
st.title("Agentic AI Chat Interface")

# Create a text input field for user messages
user_input = st.text_input("Enter your message:")

# Add a button to send the message
send_button = st.button("Send")

# Check if the button is clicked and the input is not empty
if send_button and user_input:
    # Display the user's input
    st.info(f"You entered: {user_input}")
"""

with open("app.py", "w") as f:
    f.write(streamlit_code)

# Run the Streamlit application
!streamlit run app.py & npx colab-tunnel 8501



⠙⠹⠸⠼⠴
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.225.201.163:8501

⠦⠧⠇⠏npm error code E404
⠏npm error 404 Not Found - GET https://registry.npmjs.org/colab-tunnel - Not found
⠏npm error 404
⠏npm error 404  'colab-tunnel@*' is not in this registry.
⠏npm error 404
⠏npm error 404 Note that you can also install from a
⠏npm error 404 tarball, folder, http url, or git url.
⠏npm error A complete log of this run can be found in: /root/.npm/_logs/2025-09-02T23_13_48_958Z-debug-0.log
⠏  Stopping...


In [45]:
%%writefile app.py
import streamlit as st
import sys
import os

# Add the parent directory to the sys.path to import functions from Colab cells
# This assumes the functions like generate_blog and post_to_linkedin
# are defined in the main notebook environment or a file that can be imported.
# In a real-world app, these would typically be structured differently.
# For demonstration in Colab, we'll assume they are accessible or re-defined here.

# --- Re-define necessary functions or import from notebook context ---
# This is a simplified approach for demonstration in Colab.
# In a production app, you would structure your code differently for imports.

# Assuming the OpenAI client and helper functions are needed
# We'll re-initialize the client here for the Streamlit app process
# In a more robust app, you might pass the client or use a global/singleton
try:
    from openai import OpenAI
    from google.colab import userdata
    # Attempt to get the API key - this might not work directly in the Streamlit subprocess
    # A more robust solution would involve passing the key or using env vars accessible to the subprocess
    # For this example, we'll assume the key is accessible or handle the error.
    # Let's add a note that this needs proper key handling in a real app.
    OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") or (userdata.get("OPENAI_API_KEY") if 'userdata' in sys.modules else None)

    if OPENAI_API_KEY:
         client = OpenAI(api_key=OPENAI_API_KEY)
         # Define the blog generation function (copied from cell ZDzUZtD5G57i)
         def generate_blog(content_idea: str, openai_client):
             """Generates a LinkedIn blog post using OpenAI."""
             prompt = f"""
             You are an expert LinkedIn content writer.
             Write a professional, engaging LinkedIn blog post from the following idea:
             {content_idea}

             Make sure to:
             - Start with a hook
             - Use short paragraphs
             - Add bullet points where helpful
             - Include a clear call-to-action
             - Suggest relevant hashtags
             """
             try:
                 response = openai_client.chat.completions.create(
                     model="gpt-4o",  # using GPT-4o
                     messages=[{"role": "user", "content": prompt}],
                     temperature=0.7
                 )
                 return response.choices[0].message.content
             except Exception as e:
                 st.error(f"Error generating blog: {e}")
                 return None

         # Define the placeholder LinkedIn posting function (copied from cell ZDzUZtD5G57i)
         # NOTE: This still needs a valid ACCESS_TOKEN and proper LinkedIn API call
         def post_to_linkedin(blog_text: str):
             """
             Placeholder or actual LinkedIn API Posting.
             Requires an Access Token with w_member_social scope.
             """
             # In a real scenario, you would need to retrieve or have access to the ACCESS_TOKEN here.
             # This is a significant challenge in a simple Streamlit Colab setup.
             # For now, we'll simulate or add a clear placeholder.
             st.warning("LinkedIn API posting is currently simulated.")
             st.write("Simulating LinkedIn Post...")
             st.write("Post Content:")
             st.write(blog_text[:500] + "...") # preview first 500 chars
             st.info("[LinkedIn API credentials and valid Access Token required to post automatically]")

             # --- Placeholder for actual LinkedIn API call ---
             # You would need:
             # - ACCESS_TOKEN (obtained from the OAuth flow)
             # - Your LinkedIn Person URN (obtained from /v2/me endpoint using the ACCESS_TOKEN)
             # - The POST_URL = "https://api.linkedin.com/v2/ugcPosts"
             #
             # Example (commented out):
             # if ACCESS_TOKEN: # Assuming ACCESS_TOKEN is somehow available
             #     try:
             #         # Get Author URN (example - would need to be done once and stored)
             #         # profile_url = "https://api.linkedin.com/v2/me"
             #         # headers = {"Authorization": f"Bearer {ACCESS_TOKEN}", "X-Restli-Protocol-Version": "2.0.0"}
             #         # profile_response = requests.get(profile_url, headers=headers)
             #         # profile_response.raise_for_status()
             #         # profile_data = profile_response.json()
             #         # author_urn = f"urn:linkedin:person:{profile_data.get('id')}"
             #
             #         # post_data = { ... construct post data with author_urn and blog_text ... }
             #         # headers = {"Authorization": f"Bearer {ACCESS_TOKEN}", "Content-Type": "application/json", "X-Restli-Protocol-Version": "2.0.0"}
             #         # post_response = requests.post(POST_URL, json=post_data, headers=headers)
             #         # post_response.raise_for_status()
             #         # st.success("Blog post published successfully!")
             #     except Exception as e:
             #         st.error(f"Error publishing blog post: {e}")
             # else:
             #      st.error("Access Token not available. Cannot post to LinkedIn.")

    else:
        st.error("OpenAI API key not found. Please set OPENAI_API_KEY in Colab secrets or environment variables.")
        generate_blog = None # Disable blog generation if client not initialized
        post_to_linkedin = None # Disable posting if client not initialized

except ImportError as e:
    st.error(f"Required library not found: {e}. Please ensure all necessary libraries are installed.")
    generate_blog = None
    post_to_linkedin = None


# --- Streamlit UI Layout ---
st.title("Agentic AI LinkedIn Blog Generator")

st.write("Enter your blog post idea below and generate content.")

# Text area for user to input their blog idea
user_input = st.text_area("Blog Idea", height=150, key="blog_idea_input")

# Use Streamlit's session state to store the generated blog post
if 'generated_blog' not in st.session_state:
    st.session_state.generated_blog = None

# Button to trigger blog generation
if st.button("Generate Blog"):
    if user_input and generate_blog:
        st.info("Generating blog...")
        # Call the blog generation function
        st.session_state.generated_blog = generate_blog(user_input, client)
        if st.session_state.generated_blog:
             st.success("Blog generated!")
    elif not user_input:
        st.warning("Please enter a blog idea before generating.")


# Display the generated blog post if it exists in session state
if st.session_state.generated_blog:
    st.subheader("Generated Blog Preview:")
    st.markdown(st.session_state.generated_blog)

    # Add a button to post to LinkedIn (only visible if a blog is generated)
    if post_to_linkedin and st.button("Post to LinkedIn"):
        # Call the LinkedIn posting function (currently simulated or requires token)
        post_to_linkedin(st.session_state.generated_blog)
else:
    st.info("Enter an idea above and click 'Generate Blog' to see a preview.")


# Optional: Add a section for status or logs
# st.sidebar.subheader("Status")
# st.sidebar.write("App is running.")

Overwriting app.py


In [49]:
# Run the Streamlit application in the background and expose it with ngrok
from pyngrok import ngrok
import subprocess
import time

# Kill any existing ngrok tunnels
print("Killing existing ngrok tunnels...")
ngrok.kill()
print("Existing ngrok tunnels killed.")

# Start the Streamlit app in the background
# Use subprocess.Popen to keep it running
print("Starting Streamlit app...")
process = subprocess.Popen(["streamlit", "run", "app.py"])

# Give Streamlit a moment to start
time.sleep(5)

# Expose the Streamlit port (default is 8501)
try:
    streamlit_public_url = ngrok.connect(8501)
    print(f"Streamlit app running at: {streamlit_public_url}")
    print("Click the ngrok URL above to access your Streamlit app.")

except Exception as e:
    print(f"Error starting ngrok tunnel for Streamlit: {e}")
    print("Please ensure ngrok is authenticated and no other sessions are running.")

# Keep the notebook alive (optional, but helpful for long-running apps)
# from google.colab import output
# output.eval_js('google.colab.kernel.proxyPort(8501)')

# Note: To stop the Streamlit app and ngrok tunnel, you might need to
# interrupt the Colab cell execution or run ngrok.kill() in a new cell.

Killing existing ngrok tunnels...
Existing ngrok tunnels killed.
Starting Streamlit app...
Streamlit app running at: NgrokTunnel: "https://7ad8b9805dbf.ngrok-free.app" -> "http://localhost:8501"
Click the ngrok URL above to access your Streamlit app.
